# Tradução

Uma das forças motrizes que possibilitaram o desenvolvimento da civilização humana é a capacidade de se comunicar uns com os outros. Na maioria das atividades humanas, a comunicação é a chave.

![Um robô multilíngue](./images/translation.jpg)

A inteligência artificial (IA) pode ajudar a simplificar a comunicação ao traduzir texto ou fala entre idiomas, ajudando a remover barreiras da comunicação entre países e culturas.

## Criar um recurso dos Serviços Cognitivos

No Azure, você pode usar os Serviços Cognitivos para fazer a tradução entre vários idiomas.

Caso você ainda não tenha um, use as etapas a seguir para criar um recurso dos **Serviços Cognitivos** na sua assinatura do Azure:

> **Observação**: Se você já tiver um recurso dos Serviços Cognitivos, abra a página de **Início Rápido** no portal do Azure e copie a respectiva chave e o ponto de extremidade para a célula abaixo. Caso contrário, siga as etapas abaixo para criar um.

1. Em outra guia do navegador, abra o portal do Azure em https://portal.azure.com, entrando com sua conta Microsoft.
2. Clique no botão **&#65291;Criar um recurso**, procure *Serviços Cognitivos* e crie um recurso dos **Serviços Cognitivos** com as configurações abaixo:
    - **Assinatura**: *sua assinatura do Azure*.
    - **Grupo de recursos**: *Selecione ou crie um grupo de recursos com um nome exclusivo*.
    - **Região**: *Escolha qualquer região disponível*:
    - **Nome**: *Insira um nome exclusivo*.
    - **Tipo de preço**: S0
    - **Confirmo que li e entendi os avisos**: Selecionado.
3. Aguarde até que a implantação seja concluída. Depois, entre em seu recurso dos Serviços Cognitivos e, na página **Visão geral**, clique no link para gerenciar as chaves do serviço. Você precisará do ponto de extremidade e das chaves para se conectar aos seus recursos dos Serviços Cognitivos em aplicativos clientes.

### Obter a chave e a localização do seu recurso dos Serviços Cognitivos

Para usar seu recurso dos Serviços Cognitivos, os aplicativos clientes precisam da chave de autenticação e da localização:

1. No portal do Azure, na página **Chaves e ponto de extremidade** do seu recurso dos Serviços Cognitivos, copie a **Chave 1** do recurso e cole no código abaixo, substituindo **YOUR_COG_KEY**.
2. Copie a **Localização** do recurso e cole no código abaixo, substituindo **YOUR_COG_LOCATION**.
>**Observação**: continue na página **Chave e ponto de extremidade** e copie a **Localização** desta página (por exemplo: _westus_). _Não_ adicione espaços entre as palavras no campo Localização. 
3. Execute o código abaixo clicando no botão **Executar célula** (&#9655;) à esquerda.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

## Traduzindo texto

Como o próprio nome sugere, o serviço **Tradução de texto** permite que você traduza textos de um idioma para outro.

Não há um SDK do Python para este serviço, mas você pode usar a interface REST para enviar solicitações a um ponto de extremidade por HTTP, o que é relativamente fácil de se fazer no Python usando a biblioteca de **solicitações**. As informações sobre o texto que será traduzido e o material resultante são trocadas em formato JSON.

Execute a célula a seguir para criar uma função que faça isso. Depois, teste a função com uma tradução simples do inglês para o francês.

In [ ]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

O serviço deverá ter traduzido o texto em inglês "Hello" para "Bonjour", em francês.

Note que os idiomas são especificados usando um sistema padrão de abreviações, com *en* para inglês e *fr* para francês. Você também pode usar abreviações que incluem culturas específicas, recurso útil quando o mesmo idioma é usado em áreas geográficas diferentes, em geral, com regras de ortografia diferentes. Por exemplo, *en-US* indica inglês dos Estados Unidos, enquanto *en-GB* indica inglês da Grã-Bretanha.

Execute a célula a seguir para traduzir entre inglês britânico e italiano.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

Vamos testar outra tradução, desta vez, de inglês norte-americano para chinês.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

## Tradução de Fala

Você pode usar o serviço de **Fala** para traduzir linguagem falada.

Agora, execute a célula abaixo para criar e testar uma função que usa o SDK de Fala para traduzir fala audível.

> **Observação**: Você precisará de alto-falantes para ouvir o áudio.

In [ ]:
from playsound import playsound 

# Create a function to translate audio in one language to text in another
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure the speech translation service
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure audio input
    if audio_file is None:
        audio_config = AudioConfig() # Use default input (microphone)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use file input

    # Create a translation recognizer and use it to translate speech input
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # Did we get it?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # rturn the translation
    return speech_text, translation
    

# Test the function
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
playsound(file_path)
print(result)

Observe que o idioma de destino precisa ser identificado por um código de idioma de dois caracteres (por exemplo, *en*), enquanto o idioma de origem deve incluir o indicador de cultura (por exemplo, *en-US*).

Vamos tentar traduzir do francês para o inglês.

In [ ]:
from playsound import playsound
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
playsound(file_path)
print(result)

## Saiba mais

Saiba mais sobre a [Tradução de Texto](https://docs.microsoft.com/azure/cognitive-services/translator/) e a [tradução com o serviço de Fala](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation) na documentação do serviço.